# **ETL Metadata-sitios**
---

In [9]:
#Librerias a utilizar
from google.colab import drive
import pandas as pd
import json
import pyarrow.parquet as pq


# Establece conexión con google Drive
drive.mount('/content/drive')



def aplicar_ETL():
  # Variables donde se almacenan las rutas de los archivos crudos
  metadata_9 ='/content/drive/MyDrive/Proyecto Final/Google Maps/metadata-sitios/9.json'
  metadata_11 ='/content/drive/MyDrive/Proyecto Final/Google Maps/metadata-sitios/11.json'
  metadata_10 ='/content/drive/MyDrive/Proyecto Final/Google Maps/metadata-sitios/10.json'
  metadata_6 ='/content/drive/MyDrive/Proyecto Final/Google Maps/metadata-sitios/6.json'

  rutas=[metadata_6,metadata_9,metadata_10,metadata_11]
  # Desarrollo de ETL---------------------------------
  for ruta in rutas:

    try:
      metadata_final=[]
      #  Lee archivo y devuelve un dataframe de pandas
      metadata=pd.read_json(metadata_9,lines=True)
      # Eliminación de columna importantes
      df_metadata=metadata.drop(columns=['price', 'hours', 'MISC',	'state',	'relative_results', 'url'])
      # Lista de estados (utilizando regex)
      lista_estados= ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DE", "FL", "GA",
          "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD",
          "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ",
          "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC",
          "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]
      regex = '|'.join(lista_estados)
      # Crear columna estados a partir de columna "address"
      df_metadata['state']=metadata['address'].str.extract(r'.*?(\b(?:{})\b).*'.format(regex))
      # Filtrar por los estados seleccionados
      estados=['PA','FL','CA','IL']
      df_metadata=df_metadata[df_metadata['state'].isin(estados)].reset_index(drop=True)
      # Elimina valores nulos
      df_metadata=df_metadata.dropna().reset_index(drop=True)
      # Cambio de Tipos
      df_metadata['latitude']=df_metadata['latitude'].astype('float')
      df_metadata['longitude']=df_metadata['longitude'].astype('float')

      # Trabajaremos con los negocios que contengan mas de 20 reseñas, ya que menos reseñas no seria eficiente para el sistema de recomendación
      df_metadata=df_metadata[df_metadata['num_of_reviews']>20].reset_index(drop=True)
      # Adaptar a formato string la columna "category"
      df_metadata['category']=df_metadata['category'].apply(str).str.strip('[').str.strip(']')
      # Filtrado de negocios por rubro gastronómico
      palabras_clave=["Restaurant","Hamburger restaurant","Restaurant, Bar","Bar","Restaurants","Food","Bubble Tea","Coffee & Tea","Bakery","Barbecue","Buffets","Burgers","Cafes","Catering","Cocktail Bars","Desserts","Diners","Food Trucks","Ice Cream & Frozen Yogurt","Juice Bars & Smoothies","Pizza","Seafood","Steakhouses","Sushi Bars","Tapas Bars","Wine Bars","Vegetarian","Vegan","Gluten-Free"]
      # Transformar columna "category" en string
      filtro_gastronomico=df_metadata['category'].str.contains('|'.join(palabras_clave),case=False)
      df_metadata=df_metadata[filtro_gastronomico].reset_index(drop=True)
      metadata_final.append(df_metadata)



    except: pass
  df_final=pd.concat(metadata_final).reset_index(drop=True)
    # Exportar en formato parquet
  df_final.to_parquet('metadata_procesado.parquet')

    # # Muestra datasets ya procesados
  print('\n---------------------------------------------------------------------Dataset procesado-------------------------------------------------------\n')
  display(df_final)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# EJECUTA ETL
aplicar_ETL()


---------------------------------------------------------------------Dataset procesado-------------------------------------------------------



,name,address,gmap_id,description,latitude,longitude,category,avg_rating,num_of_reviews,state
0,Pizza Hut,"Pizza Hut, 14431 Culver Dr, Irvine, CA 92604",0x80dcdc5fac2a2035:0x1b43e165067c8544,Family-friendly chain known for its made-to-or...,33.706555,-117.788129,"'Pizza restaurant', 'Chicken wings restaurant'...",2.8,78,CA
1,Giuseppes Pizzeria,"Giuseppes Pizzeria, 73850 CA-111 a, Palm Deser...",0x80dafe7cf3b9d4c5:0x55c34fadbcbc4d80,Smoked meats are paired with craft brews & cla...,33.721833,-116.376333,"'Pizza restaurant', 'Italian restaurant'",4.3,45,CA
2,Chick-fil-A,"Chick-fil-A, 400 E 2nd St Soltz Residence Hall...",0x89c5877b147c5149:0xe6ff5ec36cd61f8d,"Fast-food chain serving chicken sandwiches, st...",41.006429,-76.448820,"'Fast food restaurant', 'Breakfast restaurant'...",4.2,103,PA
3,Subway,"Subway, 425 E Avenida De Los Arboles, Thousand...",0x80e83022ee661153:0x5bb017090549dcc4,Casual counter-serve chain for build-your-own ...,34.218985,-118.869612,"'Sandwich shop', 'Caterer', 'Fast food restaur...",3.9,26,CA
4,Bella Rosa,"Bella Rosa, 4046 W Skippack Pike, Skippack, PA...",0x89c699a5bbf34d65:0xb0aec48423f33c4c,"Low-key, Ferrari-themed eatery at Hotel Fiesol...",40.224112,-75.401193,"'Italian restaurant', 'Bar'",4.6,48,PA
...,...,...,...,...,...,...,...,...,...,...
2318,Timeless - An MHK Eatery,"Timeless - An MHK Eatery, 90 Tamiami Trail N, ...",0x88dae1998c55ab59:0xca17304e181716b9,Chef-driven American small plates & pizzas pai...,26.148298,-81.795414,'American restaurant',4.3,158,FL
2319,Dragon City Bakery & Cafe,"Dragon City Bakery & Cafe, 2367 Mission St, Sa...",0x808f7e3c72d32ecf:0x1d763751efbe1d8b,"Tiny bakery for pastries, wedding cakes, bread...",37.759127,-122.418956,"'Bakery', 'Cafe', 'Chinese restaurant', 'Resta...",4.1,36,CA
2320,Just Fish & Grits,"Just Fish & Grits, 3300 N Pace Blvd #50, Pensa...",0x8890bf7407784bd7:0x99f22294e687e218,"Low-key joint dishing up fried tilapia, halibu...",30.448096,-87.238088,'Seafood restaurant',4.7,298,FL
2321,D Cuisine,"D Cuisine, 2723 N Clark St, Chicago, IL 60614",0x880fd3d0fe9ec503:0x7a7fb5027c23bebd,Trendy spot for creative Chinese dim sum & lar...,41.932084,-87.644375,'Chinese restaurant',4.5,138,IL
